In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [3]:
!pip install transformers datasets sacrebleu sacremoses -q


In [4]:
!pip install evaluate -q

In [5]:
import transformers
from transformers import MarianMTModel, MarianTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
import pandas as pd
import evaluate
import numpy as np
import torch


In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

# Loading the dataset
df = pd.read_csv('/content/drive/MyDrive/Seq2Seq/balanced_data.csv')
df = df[['english', 'spanish']].dropna()


df = df.rename(columns={'english': 'translation_en', 'spanish': 'translation_es'})

# Creating Dataset_Dictionary
dataset = Dataset.from_pandas(df[['translation_en', 'translation_es']])
dataset = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': dataset['train'],
    'validation': dataset['test']
})


In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-es"

# Loading tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["translation_en"],
        truncation=True,
        padding=True
    )

    # Tokenize targets
    labels = tokenizer(
        text_target=examples["translation_es"],
        truncation=True,
        padding=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [9]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/112843 [00:00<?, ? examples/s]

Map:   0%|          | 0/12539 [00:00<?, ? examples/s]

In [ ]:
def translate(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Generate output tokens
    translated_tokens = model.generate(**inputs)

    # Decode tokens to strng
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text


In [ ]:
# examples = [
#     "hello, how are you?",
#     "i am learning deep learning",
#     "good night and sweet dreams",
#     "where is the library?",
#     "the cat is on the table",
#     "I am Pikachu"
# ]

# for sentence in examples:
#     print(f"{sentence} -> {translate(sentence)}")


In [19]:
import evaluate
bleu = evaluate.load("sacrebleu")


In [20]:
def generate_translation_batch(batch):
    inputs = tokenizer(batch["translation_en"], return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(model.device)

    # Generate translations
    translated = model.generate(input_ids)

    # Decode
    batch["predictions"] = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return batch


In [ ]:
subset = tokenized_dataset["validation"].select(range(100))  # Testing on first 100 samples

# Adding original texts back (not tokenized)
subset = dataset["validation"].select(range(100)).map(generate_translation_batch, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Prepare predictions and references
preds = subset["predictions"]
refs = [[ref] for ref in subset["translation_es"]]  # sacrebleu needs nested list

# Computing BLEU
results = bleu.compute(predictions=preds, references=refs)
print(f"BLEU Score on validation subset without fine-tuning the model: {results['score']:.2f}")


BLEU Score on validation subset: 62.46
